# Replace singe functions in tax-benefit simulations

In [1]:
import numpy as np
import pandas as pd
from functools import partial
from unittest.mock import patch

### Minimal Tax Benefit System

In [2]:
def taxes(income, tax_rate):
    return income * tax_rate

def benefits(income, consumption_floor):
    return np.clip(consumption_floor - income, 0, np.inf)
    
def available_income(income, tax_rate, consumption_floor, cutoffs):
    t = taxes(income, tax_rate)
    b = benefits(income, consumption_floor, cutoff)
    return income - t + b 

### Goal

We might want to replace the `taxes` function by something completely different but keep the rest of the code intact. 

### Example

In [3]:
def piecewise_taxes(income, tax_rates, cutoffs):
    cutoffs = np.hstack([0, cutoffs, np.inf])
    tax = 0.0
    for c, cutoff in enumerate(cutoffs[:-1]):
        if cutoff <= income and cutoffs[c + 1] <= income:
            tax += (cutoffs[c + 1] - cutoff) * tax_rates[c]
        elif cutoff <= income and cutoffs[c + 1] > income:
            tax += (income - cutoff) * tax_rates[c]
        else:
            pass
    return tax

### Problems

1. Function signatures are specific to current tax system
2. Need to patch one function (e.g. taxes) while keeping the rest intact

### 1) Make function signatures independent of current tax system

#### 1a) prescribe minimal interface

In [4]:
def taxes(income, params):
    tax_rate = params['tax_rate']
    return income * tax_rate

def benefits(income, params):
    cutoff = params['cutoff']
    consumption_floor = params['consumption_floor']
    if income < cutoff:
        return consumption_floor
    else:
        return 0
    
def available_income(income, tax_params, benefit_params):
    t = part_taxes(income, tax_params)
    b = part_benefits(income, benefit_params)
    return income - t + b
    


#### 1b) partial all functions to a minimal interface (only income)

In [5]:
def taxes(income, tax_rate):
    return income * tax_rate

def benefits(income, consumption_floor):
    return np.clip(consumption_floor - income, 0, np.inf)


part_taxes = partial(taxes, tax_rate=0.2)
part_benefits = partial(benefits, consumption_floor=5000)

def available_income(income):
    t = part_taxes(income)
    b = part_benefits(income)
    return income - t + b

### 2) Patch context manager

In [6]:
available_income(20000)

16000.0

In [7]:
part_piecewise_taxes = partial(piecewise_taxes, tax_rates=[0.2, 0.3, 0.4, 0.5], cutoffs=[10000, 20000, 30000])
with patch('__main__.part_taxes', side_effect=part_piecewise_taxes):
    print(available_income(25000))

18000.0


In [8]:
available_income(20000)

16000.0

### Further Challenges

1. Neither the partial nor the dictionary interface work with numba
2. Patching is too difficult for most users, we would need a nicer interface